<a href="https://colab.research.google.com/github/ShawnSchmucker/MAE-494-Fall-2022/blob/main/Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problem Formulation**

The objective function is the state $s(t) = [d(t), v(t), \theta(t), \omega(t)]^T$, where $d(t)$ is the distance of the rocket from the ground, $v(t)$ is its velocity, $\theta(t)$ is its angle from the vertical, and $\omega(t)$ is its angular velocity.

The control inputs are the rocket's vertical acceleration $a(t)$, and its angular acceleration $\alpha(t)$.

The dynamics are:

$\quad d(t+1) = d(t) + v(t) \Delta t$,

$\quad v(t+1) = v(t) + a(t) \Delta t$,

$\quad \theta(t+1) = \theta(t) + \omega(t) \Delta t$,

$\quad \omega(t+1) = \omega(t) + \alpha(t) \Delta t$,

where $\Delta t$ is the time step.

The controller is $c(t) = [a(t), \alpha(t)]^T = f_\theta(s(t))$, where $f_\theta$ is a neural network with optimized parameters $\theta$.

The loss is 0 for all time $t = 1,...,T-1$, where $T$ is the final time step. At $t = T,$ the loss $l(s(T), c(T)) = ||s(T)||^2 = d(T)^2 + v(T)^2 + \theta(T)^2 + \omega(T)^2$.

The optimization problem is:

$\quad \min \limits_{\theta} \quad ||s(T)||^2$

$\qquad \qquad d(t+1) = d(t) + v(t) \Delta t$,

$\qquad \qquad v(t+1) = v(t) + a(t) \Delta t$,

$\qquad \qquad \theta(t+1) = \theta(t) + \omega(t) \Delta t$,

$\qquad \qquad \omega(t+1) = \omega(t) + \alpha(t) \Delta t$,

$\qquad \qquad c(t) = [a(t), \alpha(t)]^T = f_\theta(s(t)), \, \forall t = 1,...,T-1$.

This formulation assumes...

In [ ]:
import numpy as np
import torch as t
import torch.nn as nn
from torch import optim
from torch.nn import utils
import matplotlib.pyplot as plt

DT = 0.1
G = 0.12
THRUST = 0.18
ROTATE = 20

class Dynamics(nn.Module):

  def __init__(self):
    super(Dynamics, self).__init__()
  
  @staticmethod
  def forward(state, action):

    dx_gravity = t.tensor([0., G * DT, 0., 0.])

    dx_thrust = t.tensor([0., THRUST * DT * action[0], 0., ROTATE * DT * action[1]])

    state = state + dx_gravity + dx_thrust
    
    step_mat = t.tensor([1., DT, 0., 0.],
                        [0., 1., 0., 0.],
                        [0., 0., 1., DT],
                        [0., 0., 0., 1.])
    
    state = t.matmul(step_mat, state)

    return state

class Controller(nn.Module):

  def __init__(self, dim_input, dim_hidden, dim_output):

    super(Controller self).__init__()
    self.network = nn.Sequential(
        nn.Linear(dim_input, dim_hidden),
        nn.Tanh(),
        nn.Linear(dim_hidden, dim_output),
        nn.Sigmoid()
    )
  
  def forward(self, state):
    action = self.network(state)
    return action